YA QUE AL USAR HIPERPARAMETROS EL CÓDIGO LE ECIGIÓ MUCHO AL ENTORNO DE GOOGLE COLAB SE OPTÓ POR ADECUARLO Y CORRERLO EN UN ENTORNO LOCAL CON MAS CAPACIDAD Y EL CÓDIGO ES EL SIGUEINTE:

In [ ]:
# =============================================================
# ESTA SOLUCIÓN ES PARA EJECUTARSE EN LOCAL, EN EL PC PERSONAL O
# EN UN EDITOR COMO SPYDER.
# =============================================================
# IMPORTAR LIBRERÍAS
# =============================================================

import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# =============================================================
# CARGAR ARCHIVOS
# =============================================================

# Ruta base (Debe cambiar este enlace por la ruta en donde tiene guardado los documentos en su computador personal)
ruta_base = 'C:\\Users\\fesal\\Downloads\\udea-ai-4-eng-20251-pruebas-saber-pro-colombia (1)\\'

df_train = pd.read_csv(ruta_base + 'train.csv', engine='python', on_bad_lines='skip')
df_test = pd.read_csv(ruta_base + 'test.csv', engine='python', on_bad_lines='skip')
submission_example = pd.read_csv(ruta_base + 'submission_example.csv')

# =============================================================
# LIMPIEZA Y PROCESAMIENTO DE DATOS
# =============================================================

# Eliminar columna no útil si existe
for df in [df_train, df_test]:
    if "ESTU_PRIVADO_LIBERTAD" in df.columns:
        del df["ESTU_PRIVADO_LIBERTAD"]

columns_to_convert = [
    'ESTU_PRGM_ACADEMICO','ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
    'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
    'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
    'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE'
]

# Normalizar texto eliminando tildes
df_train['ESTU_PRGM_ACADEMICO'] = df_train['ESTU_PRGM_ACADEMICO'].apply(lambda x: unidecode(str(x)))
df_test['ESTU_PRGM_ACADEMICO'] = df_test['ESTU_PRGM_ACADEMICO'].apply(lambda x: unidecode(str(x)))

# Rellenar NaNs con la moda
for col in columns_to_convert:
    mode_val = df_train[col].mode()[0]
    df_train[col] = df_train[col].fillna(mode_val)
    df_test[col] = df_test[col].fillna(mode_val)

# Codificar features
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
df_train[columns_to_convert] = encoder.fit_transform(df_train[columns_to_convert])
df_test[columns_to_convert] = encoder.transform(df_test[columns_to_convert])

# Codificar target
y_train_encoded, target_labels = pd.factorize(df_train["RENDIMIENTO_GLOBAL"])

# =============================================================
# VARIABLES X e Y
# =============================================================

X_train = df_train.drop(["ID", "RENDIMIENTO_GLOBAL"], axis=1)
y_train = y_train_encoded
X_test = df_test.drop(["ID"], axis=1)

# =============================================================
# GRID SEARCH PARA XGBOOST
# =============================================================

print("\n🔍 Buscando mejores hiperparámetros para XGBoost...")
xgb_param_grid = {
    'n_estimators': [100, 300],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0]
}

xgb = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss"
)

xgb_grid = GridSearchCV(
    estimator=xgb,
    param_grid=xgb_param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

xgb_grid.fit(X_train, y_train)

print("✅ Mejor configuración para XGBoost:", xgb_grid.best_params_)
print("✅ Mejor accuracy promedio:", xgb_grid.best_score_)

# =============================================================
# GRID SEARCH PARA GradientBoosting
# =============================================================

print("\n🔍 Buscando mejores hiperparámetros para GradientBoosting...")
gb_param_grid = {
    'n_estimators': [100, 300],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0]
}

gb = GradientBoostingClassifier(random_state=42)

gb_grid = GridSearchCV(
    estimator=gb,
    param_grid=gb_param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

gb_grid.fit(X_train, y_train)

print("✅ Mejor configuración para GradientBoosting:", gb_grid.best_params_)
print("✅ Mejor accuracy promedio:", gb_grid.best_score_)

# =============================================================
# ENTRENAMIENTO FINAL CON MEJORES PARÁMETROS
# =============================================================

# XGBoost entrenado completo
best_xgb = XGBClassifier(
    **xgb_grid.best_params_,
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss"
)
best_xgb.fit(X_train, y_train)
pred_xgb = best_xgb.predict(X_test)
pred_xgb_labels = [target_labels[i] for i in pred_xgb]

# GradientBoosting entrenado completo
best_gb = GradientBoostingClassifier(
    **gb_grid.best_params_,
    random_state=42
)
best_gb.fit(X_train, y_train)
pred_gb = best_gb.predict(X_test)
pred_gb_labels = [target_labels[i] for i in pred_gb]

# =============================================================
# CREAR CSV DE SUBMISSION
# =============================================================

# XGBoost
submission_xgb = submission_example.copy()
submission_xgb["RENDIMIENTO_GLOBAL"] = pred_xgb_labels
submission_xgb.to_csv("submission_XGBoost_GridSearch.csv", index=False)
print("✅ Archivo creado: submission_XGBoost_GridSearch.csv")

# GradientBoosting
submission_gb = submission_example.copy()
submission_gb["RENDIMIENTO_GLOBAL"] = pred_gb_labels
submission_gb.to_csv("submission_GradientBoosting_GridSearch.csv", index=False)
print("✅ Archivo creado: submission_GradientBoosting_GridSearch.csv")

print("\n🎉 Proceso completado correctamente.")
